In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import time
import re 
import random
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import text 
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.io import read_image
from pathlib import Path
from PIL import Image
from torch.nn.functional import mse_loss as mse
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as tdata
import torchvision
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import pickle 
from datetime import datetime
import torchvision.models as models
from kornia.losses import SSIMLoss, PSNRLoss
from kornia.metrics import psnr, ssim
import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter
torch.set_printoptions(linewidth=120)
from torchsummary import summary
import json
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

torch.manual_seed(1996)
torch.cuda.manual_seed(1996)
np.random.seed(1996)
random.seed(1996)

# Load the required models

In [ ]:
class costumeDataset(Dataset):
    """
    data_path: path of all the images as a list, each element in the list is path of an image
    label_map: dataframe of corresponding labels that is loaded: maps label to image-id
    class_map: a dictionary of classes: maps labels to encoded labels
    patient_map: a dataframe that maps patient_id to frequency of each label among the images of that patient
    label_col: the column in label file where the labels are saved: here is diagnosis
    transform: the type of transform. As default it is none, meaning that the images will only be transformed to tensors
    it can also have augmentations
    noisy_transform: the type of noise implemented in the image. default: none, no noisy data
    val_split: val_train ratio. default: None, no validation split.
    
    Return Dataset class representing our data set
    """
    def __init__(self, 
                 data_dir, 
                 label_map, 
                 class_map, 
                 #patient_map,
                 label_col, 
                 augment = None, 
                 noisy_transform= None,
                 image_size = (240,240)
                ):
        
        self.data_dir = data_dir
        self.label_map = label_map 
        self.augment = augment
        self.noisy_transform = noisy_transform
        self.class_map = class_map
        self.label_col = label_col
        self.image_size = image_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),
            transforms.Resize(self.image_size),
            transforms.ToTensor()
        ])
        
    def __len__(self): 
        '''returns size of the dataset'''
        return len(self.label_map)

    def __getitem__(self, index): 
        'Generates one sample of data'  
        img_path = self.data_dir+"/"+self.label_map.loc[index, self.label_col]
        img = Image.open(img_path)
        label = self.class_map[self.label_map.loc[index, "label"]]
  
        if self.transform: 
            image = self.transform(img)       
        if self.augment: 
            augment_image = self.augment(image)
            if self.noisy_transform: #here it is a mix of all five transforms
                noisy_image = self.noisy_transform(augment_image)
                return augment_image, noisy_image, label

        else: 
            if self.noisy_transform: #here it is a mix of all five transforms
                noisy_image = self.noisy_transform(image)
                return image, noisy_image, label
            
                
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.conv1 = nn.Conv2d(3, 32, 4, stride=2, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 4, stride=2, padding=1) 
    
        # Decoder
        self.deconv4 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(128*2, 64, 4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(64*2, 32, 4, stride=2, padding=1)    
        self.deconv1 = nn.ConvTranspose2d(32*2, 3, 4, stride=2, padding=1)

        self.dropout = nn.Dropout(p=0.15)
        self.act_fn = nn.LeakyReLU(negative_slope=0.2)
        self.out_fn = nn.Sigmoid()
        
    def forward(self, x):      
        #AE architecture
        # Encoder
        z1 = self.conv1(x)       
        z1 = self.act_fn(z1)
        z1 = self.dropout(z1)  
        
        z2 = self.conv2(z1)        
        z2 = self.act_fn(z2)
        z2 = self.dropout(z2)
        
        z3 = self.conv3(z2)      
        z3 = self.act_fn(z3)
        z3 = self.dropout(z3)
        
        z4 = self.conv4(z3)      
        z = self.act_fn(z4) 
        # Decoder
        x_hat = self.deconv4(z)
        x_hat = self.act_fn(x_hat)
        x_hat = torch.cat((x_hat,z3),1)  
        x_hat = self.deconv3(x_hat)
        x_hat = self.act_fn(x_hat)
        x_hat = torch.cat((x_hat,z2),1)   
        x_hat = self.deconv2(x_hat)
        x_hat = self.act_fn(x_hat)
        x_hat = torch.cat((x_hat,z1),1)     
        x_hat = self.deconv1(x_hat)
        x_hat = self.out_fn(x_hat)
        return {'z': z, 'x_hat': x_hat}
    
"_______________set experiment__________________"
def set_experiment(model, params):
    # Transform
    composed = {
        'augment': transforms.Compose([
            transforms.ToPILImage(),
            #transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomAffine(degrees=(10), scale=(0.8,1.2)),
            transforms.ToTensor()
            ]),
        
        'noisy': transforms.Compose([
            transforms.ToPILImage(),            
            transforms.RandomApply([transforms.ColorJitter(brightness=[params["brightness"], params["brightness"]])],p=params["p"]),
            transforms.ToTensor()
            ])
    }
    data_dir = "ISIC/data"
    train_path = "ISIC/labels_train.csv"
    test_path = 'ISIC/labels_test.csv'
    valid_path = 'ISIC/labels_valid.csv'
    added_noise_appendix_path = "ISIC/labels_added_noise_appendix.csv"
    added_noise_path = 'ISIC/labels_added_noise.csv'
    natural_path = 'ISIC/labels_natural.csv'
    
    
    
    class_map = {"melanoma":0,"nevus":1, "pigmented benign keratosis":2, "basal cell carcinoma":3}
    label_col = "imageId"
    train_file, valid_file, test_file,added_noise_appendix_file ,added_noise_file ,natural_file  = pd.read_csv(train_path), pd.read_csv(valid_path), pd.read_csv(test_path),pd.read_csv(added_noise_appendix_path, sep=";"),pd.read_csv(added_noise_path, sep=";"),pd.read_csv(natural_path, sep=";")
    train_file = train_file.loc[:, ~train_file.columns.str.contains('Unnamed')]
    valid_file = valid_file.loc[:, ~valid_file.columns.str.contains('Unnamed')]
    test_file = test_file.loc[:, ~test_file.columns.str.contains('Unnamed')]
    
    added_noise_appendix_file = added_noise_appendix_file.loc[:, ~added_noise_appendix_file.columns.str.contains('Unnamed')]
    added_noise_file = added_noise_file.loc[:, ~added_noise_file.columns.str.contains('Unnamed')]
    natural_file = natural_file.loc[:, ~natural_file.columns.str.contains('Unnamed')]
    
    added_noise_appendix_labels = added_noise_appendix_file.label.tolist()
    added_noise_labels = added_noise_file.label.tolist()
    natural_labels = natural_file.label.tolist()
    train_dataset = costumeDataset(data_dir = data_dir, 
             label_map = train_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = composed["augment"], 
             noisy_transform= composed["noi
             image_size = params['image_size'])
    
    test_dataset = costumeDataset(data_dir = data_dir, 
             label_map = test_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])

    valid_dataset = costumeDataset(data_dir = data_dir, 
             label_map = valid_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
   
    added_noise_appendix_dataset = costumeDataset(data_dir = data_dir, 
             label_map = added_noise_appendix_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    
    added_noise_dataset = costumeDataset(data_dir = data_dir, 
             label_map = added_noise_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    
    natural_dataset = costumeDataset(data_dir = data_dir, 
             label_map = natural_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])

    

    #set data iterators
    train_loader = DataLoader(
        train_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False,
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False,
    )
    
    test_loader =  DataLoader(
        test_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False,
    )
    added_noise_appendix_loader = DataLoader(
        added_noise_appendix_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False,
    )
    added_noise_loader = DataLoader(
        added_noise_dataset, batch_size=params['batch_size'], shuffle=True, 
        num_workers=4, pin_memory=False,
    )
    natural_loader = DataLoader(
        natural_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False,
    )
    
    # Get experiment modules
    experiment_modules ={
        'augment' : composed["augment"],
        'noisy': composed["noisy"],
        'train_loader' : train_loader, 
        'valid_loader' : valid_loader,
        'test_loader': test_loader,
        'added_noise_appendix_loader': added_noise_appendix_loader,
        'added_noise_loader': added_noise_loader,
        'natural_loader':natural_loader,
        'added_noise_appendix_labels': added_noise_appendix_labels,
        'added_noise_labels': added_noise_labels,
        'natural_labels': natural_labels
    }
    return experiment_modules


# Loading The Results

In [ ]:
#P=0.1
with open('HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8.pickle', 'rb') as f:
    x = pickle.load(f)
data_items = x. items()
data_list = list(data_items)
df1 = pd.DataFrame(data_list)


# Get Learning Curves

In [ ]:
# plot learning curves
def get_learningCurve(df, b):
    epoch = df.iloc[0,1]
    train_loss = df.iloc[1,1] #ssim train
    test_loss = df.iloc[8,1] # ssim valid
    psnr_train = df.iloc[7,1]
    psnr_test = df.iloc[15,1]
    
    fig, axs = plt.subplots(1, 2,figsize=(15,10))
    axs[0].plot(epoch,train_loss, label='Train')
    axs[0].plot(epoch,test_loss, label='Test')
    axs[0].set_title('SSIM Loss', fontsize=16, fontweight="bold")
    axs[0].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[0].set_ylabel('SSIM Loss', fontsize=16, fontweight="bold")
    axs[0].legend(loc='best')

    axs[1].plot(epoch,psnr_train, label='Train')
    axs[1].plot(epoch,psnr_test, label='Test')
    axs[1].set_title('PSNR Score', fontsize=16, fontweight="bold")
    axs[1].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[1].set_ylabel('PSNR Score', fontsize=16, fontweight="bold")
    axs[1].legend(loc='best')
    fig.suptitle(f'Learning Curve on ISIC Dataset', fontsize=20, fontweight="bold")
    fig.tight_layout()
    plt.savefig(f"LearningCurve_BENCHMARK_ISIC_{b}.png")     
    plt.show()
    return 

get_learningCurve(df, 0.8)

# Get psnr and ssim of the last epoch

In [ ]:
#for different noise levels of benchmark
def get_ssim_psnr_noisy_model(model_file,b, params):
    model = torch.load(model_file)
    model.cuda()
    experiment_modules = set_experiment(model, params)
    #setup metrics
    
    
    rssimnx = 0.0
    rpsnrnx = 0.0
    rssimxh = 0.0
    rpsnrxh = 0.0
    
    model.eval()
    with torch.inference_mode():
        
        for x, noisy_x, y in experiment_modules["test_loader"]:
            x, noisy_x, y= x.cuda(), noisy_x.cuda(), y.cuda()
            
            outputs = model(noisy_x) 
            x_hat = outputs['x_hat']

            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            rssimnx += ssimnoisyx.sum(0)
            rssimxh += ssimxxhat.sum(0)
            
            rpsnrxh += psnrxxhat.sum(0)
            rpsnrnx += psnrnoisyx.sum(0)
            #visualise
            
            plt.imshow(np.transpose(x[2].to("cpu"), (1, 2, 0)))
            plt.show()
            plt.imshow(np.transpose(noisy_x[2].to("cpu"), (1, 2, 0)))
            plt.show()
            plt.imshow(np.transpose(x_hat[2].to("cpu"), (1, 2, 0)))
            plt.show()
            print("ssim of this image", ssimnoisyx[2].item(), ssimxxhat[2].item())
            print("psnr of this image", psnrnoisyx[2].item(), psnrxxhat[2].item())
                 
    n = 399
    meanssimnx = rssimnx/n
    meanssimxh = rssimxh/n
    
    meanpsnrnx = rpsnrnx/n
    meanpsnrxh = rpsnrxh/n
    return meanssimnx, meanssimxh, meanpsnrnx, meanpsnrxh



In [ ]:
# save the psnr ssim table
def save_psnr_ssim_table(model_b):
    
    shared_params = {
     'num_epochs': 80,
     'batch_size': 15,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.001, #Alfia: 0.00001
     'image_size': (240,240),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    
    model_file = f"../HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}epoch_80"

    dic = {"noise_level":[], "mean_ssim_noisy_x":[], "mean_ssim_xhat_x":[] ,"mean_psnr_noisy_x":[], "mean_psnr_xhat_x":[]}
    b_list = np.arange(round(1.0-model_b,1), round(1.0+model_b+0.2,1), 0.2)
    print("=======================================")
    print("model_file", model_file, "model_b", model_b, "Noise_Level_Range", b_list)
    for b in b_list:
        params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": round(b,1) # is noise level
            }
        params = {**shared_params, **params} 
        print("noise_level", round(b,1), params["brightness"])
        meanssimnx, meanssimxh, meanpsnrnx, meanpsnrxh = get_ssim_psnr_noisy_model(model_file, round(b,1), params)
        dic["noise_level"].append(round(b,1)) 
        dic["mean_ssim_noisy_x"].append(meanssimnx.item()) 
        dic["mean_ssim_xhat_x"].append(meanssimxh.item()) 
        dic["mean_psnr_noisy_x"].append(meanpsnrnx.item()) 
        dic["mean_psnr_xhat_x"].append(meanpsnrxh.item())
        with open(f'exp1_benchmark/MODEL{model_b}_SSIM_PSNR_TABLE_train.pickle','wb') as handle: 
            pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return dic



In [ ]:
model_b_list = [0.8]
for model_b in model_b_list:
    dic = save_psnr_ssim_table(round(model_b,1))
    

# Images and qualitative Analysis

## Added Noise report

In [ ]:
def visualise_images_added_noise_report(model_file, b_list, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(len(b_list)*2+1, shared_params["batch_size"],figsize=(15,15), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coord
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    added_noise_labels = ["Melanoma1", "Nevus", "BCC1", "PBK1", "Melanoma2", "PBK2", "BCC2"]
    
    model = torch.load(model_file)
    model.cuda()
    
    for b in range(len(b_list)):
        params = { 
            "gamma": 0.01,
            "brightness": round(b_list[b],1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_noise_loader"]))
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(noisy_x) 
            #y_hat = outputs['y_hat'].cuda()
            x_hat = outputs['x_hat']
            # qualitative scores
            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            ssim_psnr_values["ssim_noisy_x"].append(ssimnoisyx)
            ssim_psnr_values["ssim_xhat_x"].append(ssimxxhat)
            ssim_psnr_values["psnr_noisy_x"].append(psnrnoisyx)
            ssim_psnr_values["psnr_xhat_x"].append(psnrxxhat)
            
            for i in range(params["batch_size"]):
                #plot images
                axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0)), aspect='auto', origin="lower") #GT
                axs[b*2+1,i].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0)),aspect='auto', origin="lower") #noisy
                axs[b*2+2,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)),aspect='auto', origin="lower")
                

                axs[0,i].set_xticklabels([])
                axs[0,i].set_yticklabels([])
                axs[b*2+1,i].set_xticklabels([])
                axs[b*2+1,i].set_yticklabels([])
                axs[b*2+2,i].set_xticklabels([])
                axs[b*2+2,i].set_yticklabels([])
                axs[0,i].set_aspect('equal')
                axs[b*2+1,i].set_aspect('equal')
                axs[b*2+2,i].set_aspect('equal')
                
                axs[8,i].set_xlabel(added_noise_labels[i], fontweight = "bold", fontsize = 15)
                if i == 0:
                    axs[0,i].set_ylabel("GT", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=20)
                    axs[b*2+1,i].set_ylabel(f"{b_list[b]}", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=20)
                    axs[b*2+2,i].set_ylabel(f"SDAE {b_list[b]}", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=40)    
                # add ssim and psnr of each
                
                #axs[i,b*2+1].text(left, top, f'{ssimnoisyx[i]}\n{psnrnoisyx[i]}',horizontalalignment='center',verticalalignment='center',rotation=45,transform=axs[i,b*2+1].transAxes)                    
                #axs[i,b*2+2].text(left, top, f'{ssimxxhat[i]}\n{psnrxxhat[i]}',horizontalalignment='center',verticalalignment='center',rotation=45,transform=axs[i,b*2+2].transAxes)
                axs[b*2+1,i].text(x=220,y=1,s=f'{round(ssimnoisyx[i].item(),2)}\n{round(psnrnoisyx[i].item(),2)}',horizontalalignment='right',verticalalignment='bottom', fontsize=10, fontweight="demi", bbox=dict(facecolor='white', alpha=0.5))                    
                axs[b*2+2,i].text(x=220,y=1,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='right',verticalalignment='bottom', fontsize=10, fontweight="demi", bbox=dict(facecolor="white", alpha=0.5))
                #axs[i,0].set_axis_off()                  
                #axs[i,b*2+1].set_axis_off()        
                #axs[i,b*2+2].set_axis_off()        
          

    fig.suptitle('De-noising Test Images With Different Levels of Added Noise', fontsize=25, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95, bottom=0.05)
    #fig.tight_layout() # made adjust not work
    name = f"exp1_benchmark/VISUALISE_ISIC_0.8_NOISERANGE{b_list}_REPORT.png".replace(',', '_').replace("]","").replace("[","_").replace(" ","")
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values



# save the psnr ssim table
def save_images_added_report(model_b):
    
    shared_params = {
     'num_epochs': 80,
     'batch_size': 7,
     'random_seed': 1996,
     #'optimizer_name': 'adamW',
     #'learning_rate': 0.001, #Alfia: 0.00001
     'image_size': (240,240),
     #'degrees': (-10, 10),
     #'translate': (0.0, 0.5),
     #'scale': (0.5, 0.95),
     #'dropout_p': 0.5,# dropout probability
     #'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     #'alpha':1, #weight of ssim
     #'weight_decay':1e-3, # L2 regularization
     'p':1
    }
    
    model_file = f"HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}epoch_80"

    b_list = [0.2,0.6,1.4,1.8]
    #print("=======================================")
    #print("model_file", model_file, "model_b", model_b, "Noise_Level_Range", b_list)
    ssim_psnr_values = visualise_images_added_noise_report(model_file, b_list, shared_params)

    #with open(f'exp1_benchmark/MODEL{model_b}_SSIM_PSNR_TABLE.pickle','wb') as handle: 
    #    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return ssim_psnr_values
ssim_psnr_values = save_images_added_report(0.8)


## Added_ noise_ appendix

In [ ]:
def visualise_images_added_noise_appendix(model_file, b_list, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(len(b_list)*2+1,shared_params["batch_size"] ,figsize=(50,50), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    

    added_noise_labels_appendix = ["BCC1", "BCC2", "BCC3", "BCC4", "Melanoma1", "Melanoma2", "Melanoma3", "Melanoma4", "Melanoma5", "Melanoma6", "Nevus1", "PBK1", "PBK2"]
    model = torch.load(model_file)
    model.cuda()
    
    for b in range(len(b_list)):
        params = { 
            "gamma": 0.01,
            "brightness": round(b_list[b],1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_noise_appendix_loader"]))
            
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(x) 
            #y_hat = outputs['y_hat'].cuda()
            x_hat = outputs['x_hat']
            # qualitative scores
            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            ssim_psnr_values["ssim_noisy_x"].append(ssimnoisyx)
            ssim_psnr_values["ssim_xhat_x"].append(ssimxxhat)
            ssim_psnr_values["psnr_noisy_x"].append(psnrnoisyx)
            ssim_psnr_values["psnr_xhat_x"].append(psnrxxhat)
            
            for i in range(params["batch_size"]):
                #plot images
                axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #GT
                axs[b*2+1,i].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
                axs[b*2+2,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
                axs[0,i].set_xticklabels([])
                axs[0,i].set_yticklabels([])
                axs[b*2+1,i].set_xticklabels([])
                axs[b*2+1,i].set_yticklabels([])
                axs[b*2+2,i].set_xticklabels([])
                axs[b*2+2,i].set_yticklabels([])
                axs[0,i].set_aspect('equal')
                axs[b*2+1,i].set_aspect('equal')
                axs[b*2+2,i].set_aspect('equal')
                
                axs[16,i].set_xlabel(added_noise_labels_appendix[i], fontweight = "bold", fontsize = 15)
                if i == 0:
                    axs[0,i].set_ylabel("GT", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=20)
                    axs[b*2+1,i].set_ylabel(f"{b_list[b]}", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=20)
                    axs[b*2+2,i].set_ylabel(f"SDAE {b_list[b]}", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=40)    
                # add ssim and psnr of each

                axs[b*2+1,i].text(x=220,y=1,s=f'{round(ssimnoisyx[i].item(),2)}\n{round(psnrnoisyx[i].item(),2)}',horizontalalignment='right',verticalalignment='bottom', fontsize=10, fontweight="demi", bbox=dict(facecolor='white', alpha=0.5))                    
                axs[b*2+2,i].text(x=220,y=1,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='right',verticalalignment='bottom', fontsize=10, fontweight="demi", bbox=dict(facecolor="white", alpha=0.5))
                     
          

    fig.suptitle('De-noising Test Images With Different Levels of Added Noise', fontsize=50, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0)
    name = f"exp1_benchmark/VISUALISE_ISIC_0.8_NOISERANGE{b_list}_APPENDIX.png".replace(',', '_').replace("]","").replace("[","_").replace(" ","")
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values


# save the psnr ssim table
def save_images_added_appendix(model_b):
    
    shared_params = {
     'num_epochs': 80,
     'batch_size': 13,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.001, 
     'image_size': (240,240),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3, # L2 regularization
     'p':1
    }
    
    model_file = f"../HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}epoch_80"

    b_list = [0.2,0.4,0.6,0.8,1.2,1.4,1.6,1.8]
    #print("=======================================")
    #print("model_file", model_file, "model_b", model_b, "Noise_Level_Range", b_list)
    ssim_psnr_values = visualise_images_added_noise_appendix(model_file, b_list, shared_params)

    #with open(f'exp1_benchmark/MODEL{model_b}_SSIM_PSNR_TABLE.pickle','wb') as handle: 
    #    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return ssim_psnr_values
ssim_psnr_values = save_images_added_appendix(0.8)



# Appendix Images

In [ ]:
def visualise_images_added_noise_appendix(model_file, b_list, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(2,3 ,figsize=(50,40), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height

    model = torch.load(model_file)
    model.cuda()
    
    for b in range(len(b_list)):
        params = { 
            "gamma": 0.01,
            "brightness": round(b_list[b],1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_noise_appendix_loader"]))
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(x) 
            #y_hat = outputs['y_hat'].cuda()
            x_hat = outputs['x_hat']
            # qualitative scores
            
            axs[0,0].imshow(np.transpose(x[0].to("cpu"), (1, 2, 0))) #GT
            axs[b+1,0].imshow(np.transpose(x_hat[0].to("cpu"), (1, 2, 0))) #denoised
            #axs[b*2+1,i].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy

            axs[0,1].imshow(np.transpose(x[2].to("cpu"), (1, 2, 0))) #GT
            axs[b+1,1].imshow(np.transpose(x_hat[2].to("cpu"), (1, 2, 0))) #denoised

            axs[0,2].imshow(np.transpose(x[4].to("cpu"), (1, 2, 0))) #GT
            axs[b+1,2].imshow(np.transpose(x_hat[4].to("cpu"), (1, 2, 0))) #denoised
            
            for i in range(3):
                axs[0,i].set_xticklabels([])
                axs[0,i].set_yticklabels([])
                axs[b+1,i].set_xticklabels([])
                axs[b+1,i].set_yticklabels([])
                
                axs[0,i].set_aspect('equal')
               
                axs[b+1,i].set_aspect('equal')
                
                if i == 0:
                    axs[0,i].set_ylabel("GT", fontweight = "bold", fontsize = 80, rotation = 0, labelpad=60)
                    axs[1,i].set_ylabel(f"SDAE {b_list[b]}", fontweight = "bold", fontsize = 80, rotation = 0, labelpad=200)
                    

    fig.suptitle('The Removal of the Datasets Artifacts', fontsize=100, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0, top = 0.95, bottom = 0.05)
    name = f"../VISUALISE_ISIC_artifact_.png"
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values


# save the psnr ssim table
def save_images_added_appendix(model_b):
    
    shared_params = {
     'num_epochs': 80,
     'batch_size': 5,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.001, 
     'image_size': (240,240),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3, # L2 regularization
     'p':1
    }
    
    model_file = f"HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}epoch_80"

    b_list = [0.6]
    ssim_psnr_values = visualise_images_added_noise_appendix(model_file, b_list, shared_params)
    return ssim_psnr_values
ssim_psnr_values = save_images_added_appendix(0.8)



## Natural Noise

In [ ]:
def visualise_images_natural(model_file, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(2, shared_params["batch_size"] ,figsize=(25,10), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    model = torch.load(model_file)
    model.cuda()
    
    params = { 
        "gamma": 0.01,
        "brightness": 0.6 # is dummy value
        }
    params = {**shared_params, **params}
    experiment_modules = set_experiment(model, params)
    
    model.eval()
    
    
    with torch.inference_mode():
        x, noisy_x, y = next(iter(experiment_modules["natural_loader"]))
        x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
        outputs = model(x) 
        #y_hat = outputs['y_hat'].cuda()
        x_hat = outputs['x_hat']
        print(y)
        # qualitative scores
        
        #plot images
        for i in range(params["batch_size"]):
            axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #original image
            axs[1,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
            axs[0,i].set_xticklabels([])
            axs[0,i].set_yticklabels([])
            axs[1,i].set_xticklabels([])
            axs[1,i].set_yticklabels([])
            axs[0,i].set_aspect('equal')
            axs[1,i].set_aspect('equal')
            
            axs[1,i].set_xlabel(experiment_modules["natural_labels"][i], fontweight = "bold", fontsize = 25)
            if i == 0:
                axs[0,i].set_ylabel("Original Image", fontweight = "bold", fontsize = 25)
                axs[1,i].set_ylabel("Denoised Image", fontweight = "bold", fontsize = 25)

       
    fig.suptitle(f'De-noising Naturally "Bad Light" Images', fontsize=30, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0)

    name = f"../VISUALISE_ISIC_0.8_NATURAL.png"
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values


# save the psnr ssim table
def save_images_natural(model_b):
    
    shared_params = {
    
     'batch_size': 5,
     'random_seed': 1996,     
     'image_size': (240,240),     
     'n_classes': 4,
     'p':0
    }
    
    model_file = f"HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS{model_b}epoch_80"

    print("=======================================")
    print("model_file", model_file, "model_b", model_b)
    ssim_psnr_values = visualise_images_natural(model_file, shared_params)

    #with open(f'exp1_benchmark/MODEL{model_b}_SSIM_PSNR_TABLE.pickle','wb') as handle: 
    #    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return ssim_psnr_values
ssim_psnr_values = save_images_natural(0.8)



# Reconstruction in different epochs fixed noise level

In [ ]:

def visualise_images_epochs(epoch_list, b, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(shared_params["batch_size"], len(epoch_list)+2 ,figsize=(20,20), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2

    added_noise_labels = ["Melanoma", "Nevus", "BCC", "PBK"]
    for m in range(len(epoch_list)):
        model_file = f"HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8epoch_{epoch_list[m]}"
        model = torch.load(model_file)
        model.cuda()
        params = { 
            "gamma": 0.01,
            "brightness": round(b,1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_noise_loader"]))
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(noisy_x) 
            #y_hat = outputs['y_hat'].cuda()
            x_hat = outputs['x_hat']
            # qualitative scores
            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            ssim_psnr_values["ssim_noisy_x"].append(ssimnoisyx)
            ssim_psnr_values["ssim_xhat_x"].append(ssimxxhat)
            ssim_psnr_values["psnr_noisy_x"].append(psnrnoisyx)
            ssim_psnr_values["psnr_xhat_x"].append(psnrxxhat)
            
            for i in range(params["batch_size"]):
                #plot images
                axs[i,0].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #GT
                axs[i,1].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
                axs[i,m+2].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
                axs[i,0].set_xticklabels([])
                axs[i,0].set_yticklabels([])
                axs[i,1].set_xticklabels([])
                axs[i,1].set_yticklabels([])
                axs[i,m+2].set_xticklabels([])
                axs[i,m+2].set_yticklabels([])
                axs[i,0].set_aspect('equal')
                axs[i,1].set_aspect('equal')
                axs[i,m+2].set_aspect('equal')
                
                if i == params["batch_size"]+1:
                    axs[i,0].set_xlabel("GT", fontweight = "bold", fontsize = 25, rotation=45)
                    axs[i,1].set_xlabel(f"Noisy({b})", fontweight = "bold", fontsize = 25, rotation=45)
                    axs[i,m+2].set_xlabel(f"Epoch {epoch_list[m]}", fontweight = "bold", fontsize = 25, rotation=45)
                
                if m == 0: 
                    axs[i,m].set_ylabel(added_noise_labels[i], fontweight = "bold", fontsize = 25, rotation=90)
                # add ssim and psnr of each

                axs[i,m+2].text(x=220,y=220,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='right',verticalalignment='bottom', fontsize=25, fontweight="demi", bbox=dict(facecolor='white', alpha=0.5))
                axs[i,1].text(x=220,y=220,s=f'{round(ssimnoisyx[i].item(),2)}\n{round(psnrnoisyx[i].item(),2)}',horizontalalignment='right',verticalalignment='bottom', fontsize=25, fontweight="demi", bbox=dict(facecolor='white', alpha=0.5))
                

    fig.suptitle('Reconstruction at Fixed Noise Level in Different Epochs', fontsize=35, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0)
   
    name = f"exp1_benchmark/VISUALISE_ISIC_0.8_LARGEEPOCH_b{b}{epoch_list}.png".replace(',', '_').replace("]","").replace("[","_").replace(" ","")
    plt.savefig(name)     
    plt.show()
    return ssim_psnr_values

# save the psnr ssim table for different epochs
def save_images_epochs(b, epoch_list):
    
    shared_params = {
    
     'batch_size': 4,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.001, 
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3, # L2 regularization
     'p':1
    }
    
    print("=======================================")
    print("Noise_Level", b, "epoch range", epoch_list)
    ssim_psnr_values = visualise_images_epochs(epoch_list, b, shared_params)
    return ssim_psnr_values

b = 0.6
epoch_list = [10,40,80]
ssim_values = save_images_epochs(b, epoch_list)




# Loading the tables to use in the report

In [ ]:
def get_table(ssim_psnr_table_path):
    with open(ssim_psnr_table_path, 'rb') as f:
        x = pickle.load(f)

    #data_items = x.items()
    #data_list = list(data_items)
    #df = pd.DataFrame(data_list)
    df = pd.DataFrame.from_dict(x)


    #ssim with 3 decimal
    df["mean_ssim_noisy_x"] = df["mean_ssim_noisy_x"].round(2)
    df["mean_ssim_xhat_x"] = df["mean_ssim_xhat_x"].round(2)
    # psnr with 2 decimal
    df["mean_psnr_noisy_x"] = df["mean_psnr_noisy_x"].round(2)
    df["mean_psnr_xhat_x"] = df["mean_psnr_xhat_x"].round(2)
    return df

In [ ]:
#Model with 0.8
ssim_psnr_table_path = "../MODEL0.8_SSIM_PSNR_TABLE_train.pickle"
df = get_table(ssim_psnr_table_path)

In [ ]:
display(df)

# Model Analysis

In [ ]:
# https://ravivaishnav20.medium.com/visualizing-feature-maps-using-pytorch-12a48cd1e573
# visualising feature map of an image: noisy and not noisy
from mpl_toolkits.axes_grid1 import make_axes_locatable
def layer_visualise(model, experiment_modules):
    names = ["image", "conv1", "conv2", "conv3", "conv4", "deconv4", "deconv3", "deconv2", "deconv1"]
    #for x, noisy_x, y in experiment_modules["added_noise_loader"]:
    x, noisy_x, y = next(iter(experiment_modules["added_noise_loader"]))
    outputs = []
    model.eval()
    with torch.inference_mode():

        outputs.append(x)

        image1 = model.conv1(x)
        image1 = model.act_fn(image1)
        
        outputs.append(image1)
        image2 = model.conv2(image1)
        image2 = model.act_fn(image2)
        outputs.append(image2)
        image3 = model.conv3(image2)
        image3 = model.act_fn(image3)
        outputs.append(image3)
        image4 = model.conv4(image3)
        image4 = model.act_fn(image4)
        outputs.append(image4)

        image5 = model.deconv4(image4)
        image5 = model.act_fn(image5)
        outputs.append(image5)

        image6 = model.deconv3(torch.cat((image5,image5),1))
        image6 = model.act_fn(image6)
        outputs.append(image6)

        image7 = model.deconv2(torch.cat((image6,image6),1))
        image7 = model.act_fn(image7)
        outputs.append(image7)

        image8 = model.deconv1(torch.cat((image7,image7),1))
        image8 = model.out_fn(image8)
        outputs.append(image8)

    #from 3d to 2D

    processed = []
    for feature_map in outputs:
        feature_map = feature_map.squeeze(0)
        
        gray_scale = torch.sum(feature_map,0)
        gray_scale = gray_scale / feature_map.shape[0]
        processed.append(gray_scale.data.cpu().numpy())
    fig, axs = plt.subplots(2,5,figsize=(30,15))
    fig.delaxes(axs[1,4])
    for i in range(5):
        im = axs[0,i].imshow(processed[i], cmap='gray')
        divider = make_axes_locatable(axs[0,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')

        axs[0,i].axis("off")
        axs[0,i].set_title(names[i].split('(')[0], fontsize=25)
    for i in range(4):
        
        im = axs[1,i].imshow(processed[i+5], cmap='gray')
        divider = make_axes_locatable(axs[1,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        axs[1,i].axis("off")
        axs[1,i].set_title(names[i+5].split('(')[0], fontsize=25)
            
    fig.suptitle('Feature Maps of a Sample', fontsize=35, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95, bottom=0.05)
    name = f"../MODEL_ISIC.png"
    plt.savefig(name) 

    plt.show()

    return

def get_layer_visualisation():
    model_file = "HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8epoch_80"

    shared_params = {
     'num_epochs': 80,
     'batch_size': 1,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.001, #Alfia: 0.00001
     'image_size': (240,240),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": 0.6 # from 0.8 til 1.2
        }
    params = {**shared_params, **params} 
    model = torch.load(model_file)
    model.to("cpu")
    experiment_modules = set_experiment(model, params)
    layer_visualise(model, experiment_modules)
    return 

get_layer_visualisation()

# Visualising filters of conv1 and deconv1

In [ ]:
def plot_filters_single_channel(t):
    #https://github.com/Niranjankumar-c/DeepLearning-PadhAI/blob/master/DeepLearning_Materials/6_VisualizationCNN_Pytorch/CNNVisualisation.ipynb
    #kernels depth * number of kernels
    nplots = t.shape[0]*t.shape[1]
    ncols = 12
    
    nrows = 1 + nplots//ncols
    #convert tensor to numpy image
    npimg = np.array(t.numpy(), np.float32)
    
    count = 0
    fig = plt.figure(figsize=(20,15))
    print(t.shape[0],t.shape[1])
    #looping through all the kernels in each channel
    for i in range(t.shape[0]):
        for j in range(t.shape[1]):
            count += 1
            ax1 = fig.add_subplot(nrows, ncols, count)
            npimg = np.array(t[i, j].numpy(), np.float32)
            npimg = (npimg - np.mean(npimg)) / np.std(npimg)
            npimg = np.minimum(1, np.maximum(0, (npimg + 0.5)))
            if j == 0:
                ax1.imshow(npimg, cmap='Reds')
            if j == 1:
                ax1.imshow(npimg, cmap='Greens')
            if j == 2:
                ax1.imshow(npimg, cmap='Blues')
            #ax1.set_title(str(i) + ',' + str(j))
            ax1.axis('off')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
    fig.suptitle('Filters of the First Convolution', fontsize=25, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top = 0.95, bottom = 0.05)
    #fig.tight_layout() # made adjust not work
    name = f"exp1_benchmark/con_kernel_ISIC.png"
    plt.savefig(name) 

    plt.tight_layout()
    plt.show()
    return

def get_filters():
    model_file = "HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8/HPO_MAIN_BENCHMARK_CORRECTED_GAMMA0.01_P0.6_BRIGHTNESS0.8epoch_80"

    shared_params = {
     'num_epochs': 80,
     'batch_size': 1,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.001,
     'image_size': (240,240),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": 1.4
        }
    params = {**shared_params, **params} 
    model = torch.load(model_file)
    model.to("cpu")
    experiment_modules = set_experiment(model, params)
    weight_tensor = model.conv1.weight.data 
    plot_filters_single_channel(weight_tensor.cpu())

    return 
get_filters()